In [1]:
import numpy as np
from torch import tensor, float32

# pip install -U qdna-lib
from qdna.embedding import (
    NqeAeFeatureMap, 
    NqeZZFeatureMap
)

## Dataset

In [2]:
from qclib.machine_learning.datasets import digits

# Dataset load.
seed = 42

# Here you can choose the dataset classes. For example: [0,1], [1,8], [2,4], etc.
_, training_input, test_input, class_labels = digits.load(
    classes=[0, 1],
    training_size=150,
    test_size=20,
    random_seed=seed
)

feature_dim = len(training_input[class_labels[0]][0])
num_qubits = int(np.ceil(np.log2(feature_dim)))

# The `digits.load()` function does not return the data as expected by NQE.
# Because of this, we need to do a simple rearrangement as follows.

# Format the data for NQE's `fit` and `transform` functions.
train_data = np.array(
    training_input[class_labels[0]] + \
    training_input[class_labels[1]]
)

test_data = np.array(
    test_input[class_labels[0]] + \
    test_input[class_labels[1]]
)

# Target labels.
train_targets = np.array(
    [class_labels[0]] * len(training_input[class_labels[0]]) + \
    [class_labels[1]] * len(training_input[class_labels[1]])
)

test_targets = np.array(
    [class_labels[0]] * len(test_input[class_labels[0]]) + \
    [class_labels[1]] * len(test_input[class_labels[1]])
)

In [3]:
print(feature_dim)
print(num_qubits)

64
6


## Neural Quantum Embedding (NQE)

In [4]:
# Create a trainable amplitude encoding feature map
t_fmap = NqeAeFeatureMap(num_qubits, reps=1, set_global_phase=True)

# Pre-training the embedding.
t_fmap.fit(train_data, train_targets, iters=10)

# Tranforms the original dataset (using trained feature map)
train_nqe = t_fmap.transform(tensor(train_data, dtype=float32))
test_nqe = t_fmap.transform(tensor(test_data, dtype=float32))

# From now on, the trace distance of data with different labels is maximized.

Training [10%]	Loss: 0.0011	Avg. Loss: 0.0011
Training [20%]	Loss: 0.0000	Avg. Loss: 0.0005
Training [30%]	Loss: 0.0000	Avg. Loss: 0.0004
Training [40%]	Loss: 0.0000	Avg. Loss: 0.0003
Training [50%]	Loss: 0.0000	Avg. Loss: 0.0002
Training [60%]	Loss: 0.0000	Avg. Loss: 0.0002
Training [70%]	Loss: 0.0000	Avg. Loss: 0.0002
Training [80%]	Loss: 0.0000	Avg. Loss: 0.0001
Training [90%]	Loss: 0.0000	Avg. Loss: 0.0001
Training [100%]	Loss: 0.0000	Avg. Loss: 0.0001


In [5]:
print(train_nqe)

[[ 0.1649951  -0.34003606  0.12849116 ... -0.01313476 -0.06123468
  -0.31228036]
 [ 0.14164735 -0.31942713  0.11426534 ... -0.01206358 -0.06594189
  -0.30632454]
 [ 0.16070811 -0.33201817  0.13078459 ... -0.00160599 -0.06941289
  -0.31027117]
 ...
 [ 0.16053954 -0.3372686   0.11936585 ... -0.0049169  -0.06335965
  -0.32194883]
 [ 0.15774992 -0.3239718   0.11463089 ... -0.00721916 -0.06513025
  -0.30494064]
 [ 0.15700172 -0.32119817  0.11222707 ... -0.00219209 -0.06597438
  -0.30045938]]


In [6]:
print(test_nqe)

[[ 0.16798161 -0.3449557   0.13013968 ... -0.00406188 -0.06959986
  -0.30671495]
 [ 0.15224478 -0.32395446  0.11761674 ...  0.00194404 -0.06834033
  -0.30400974]
 [ 0.15287742 -0.3305541   0.11626966 ...  0.00098379 -0.07155859
  -0.3025763 ]
 ...
 [ 0.14758082 -0.30902207  0.11330259 ... -0.006437   -0.0658424
  -0.27589327]
 [ 0.16965239 -0.34798557  0.13330437 ...  0.00511582 -0.07081909
  -0.32408315]
 [ 0.15052529 -0.3271492   0.12427101 ... -0.01104955 -0.06896384
  -0.30707955]]
